In [26]:
import pandas as pd
from mido import Message, MidiFile, MidiTrack, MetaMessage
import pygame

In [27]:
df = pd.read_csv('summarised_offspring_scaffold_load.csv')
dfsum=df.sum(axis=1)

In [28]:
midIn = MidiFile('FurEliseOneStaveV2.mid')

In [29]:
MessagesInput = []
for i, track in enumerate(midIn.tracks):
    print('Track {}: {}'.format(i, track.name))
    for msg in track:
        print(str(msg))
        MessagesInput.append(str(msg))

Track 0: Piano 
<meta message track_name name='Piano\x00' time=0>
<meta message time_signature numerator=1 denominator=8 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
<meta message key_signature key='C' time=0>
<meta message set_tempo tempo=545455 time=0>
control_change channel=0 control=121 value=0 time=0
program_change channel=0 program=0 time=0
control_change channel=0 control=7 value=100 time=0
control_change channel=0 control=10 value=64 time=0
control_change channel=0 control=91 value=0 time=0
control_change channel=0 control=93 value=0 time=0
<meta message midi_port port=0 time=0>
note_on channel=0 note=76 velocity=80 time=0
note_on channel=0 note=76 velocity=0 time=113
note_on channel=0 note=75 velocity=80 time=7
note_on channel=0 note=75 velocity=0 time=113
<meta message time_signature numerator=3 denominator=8 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=7>
note_on channel=0 note=76 velocity=80 time=0
note_on channel=0 note=76 velocity=0 time=113
note_on

In [30]:
detuneFactor=-20
rowNumber=11

def myround(x, base=5):
    return base * round(x/base)

index_j=0
index_note_onset=0
MessagesOutput=[]
for j in MessagesInput:    
    if "velocity=80" in j:
        if index_note_onset>5:
            pitchbend=int(myround((df.iloc[rowNumber,index_j+2])) * detuneFactor)
            index_j=index_j+1
            MessagesOutput.append("pitchwheel pitch=" + str(pitchbend))
            print("pitchwheel pitch=" + str(pitchbend))
        index_note_onset = index_note_onset+1
    MessagesOutput.append(j)
    print(j)

<meta message track_name name='Piano\x00' time=0>
<meta message time_signature numerator=1 denominator=8 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
<meta message key_signature key='C' time=0>
<meta message set_tempo tempo=545455 time=0>
control_change channel=0 control=121 value=0 time=0
program_change channel=0 program=0 time=0
control_change channel=0 control=7 value=100 time=0
control_change channel=0 control=10 value=64 time=0
control_change channel=0 control=91 value=0 time=0
control_change channel=0 control=93 value=0 time=0
<meta message midi_port port=0 time=0>
note_on channel=0 note=76 velocity=80 time=0
note_on channel=0 note=76 velocity=0 time=113
note_on channel=0 note=75 velocity=80 time=7
note_on channel=0 note=75 velocity=0 time=113
<meta message time_signature numerator=3 denominator=8 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=7>
note_on channel=0 note=76 velocity=80 time=0
note_on channel=0 note=76 velocity=0 time=113
note_on channel=0 note=

In [31]:
test=0
mTest=0
cTest=0
pTest=0
PitchTest=0
nTest=0

stringCodeMeta=""
stringCodeNote=""
stringCodePitch=""
stringCodeProgram=""
stringCodeControl=""

midOut = MidiFile()
track = MidiTrack()
midOut.tracks.append(track)

for k in MessagesOutput:
    splat=k.split(" ")
    test=test+1
    first="'" + splat[0].replace("<","") + "'" + ", "
    second=splat[1] + ", "
    if len(splat)==3:
        third=splat[2].replace(">","")
        fourth=""
        fifth=""
        sixth=""
        seventh=""
        eighth=""
    elif len(splat)==4:
        third=splat[2].replace(">","") + ", "
        fourth=splat[3].replace(">","")
        fifth=""
        sixth=""
        seventh=""
        eighth=""
    elif len(splat)==5:
        third=splat[2].replace(">","") + ", "
        fourth=splat[3].replace(">","") + ", "
        fifth=splat[4].replace(">","")
        sixth=""
        seventh=""
        eighth=""
    elif len(splat)==6:
        third=splat[2].replace(">","") + ", "
        fourth=splat[3].replace(">","") + ", "
        fifth=splat[4].replace(">","") + ", "
        sixth=splat[5].replace(">","")
        seventh=""
        eighth=""
    elif len(splat)==7:
        third=splat[2].replace(">","") + ", "
        fourth=splat[3].replace(">","") + ", "
        fifth=splat[4].replace(">","") + ", "
        sixth=splat[5].replace(">","") + ", "
        seventh=splat[6].replace(">","")
        eighth=""
    elif len(splat)==8:
        third=splat[2].replace(">","") + ", "
        fourth=splat[3].replace(">","") + ", "
        fifth=splat[4].replace(">","") + ", "
        sixth=splat[5].replace(">","") + ", "
        seventh=splat[6].replace(">","") + ", "
        eighth=splat[7].replace(">","")

    if "meta message" in k:
        third="'" + splat[2].replace("<","") + "'" + ", "
        stringCodeMeta="track.append(MetaMessage(" + third + fourth + fifth + sixth + seventh + eighth + "))"
        exec(stringCodeMeta)
        mTest=mTest+1
    elif "control_change" in k:
        stringCodeControl="track.append(Message(" + first + second + third + fourth + fifth + "))"
        exec(stringCodeControl)
        cTest=cTest+1
    elif "program_change" in k:
        stringCodeProgram="track.append(Message(" + first + second + third + fourth + "))"
        exec(stringCodeProgram)
        pTest=pTest+1
    elif "pitchwheel" in k:
        stringCodePitch="track.append(Message(" + first + second + "))"
        exec(stringCodePitch)
        PitchTest=PitchTest+1
    elif "note_on" in k:
        stringCodeNote="track.append(Message(" + first + second + third + fourth + fifth + "))"
        exec(stringCodeNote)
        nTest=nTest+1

midOut.save('output.mid')

In [32]:
print(test, mTest, cTest, pTest, PitchTest, nTest, stringCodeMeta, stringCodeNote)

339 8 21 1 99 210 track.append(MetaMessage('end_of_track', time=1)) track.append(Message('note_on', channel=0, note=69, velocity=0, time=0))


In [33]:
midIn2 = MidiFile('output.mid')

MessagesInput2 = []
for ii, tracktrack in enumerate(midIn2.tracks):
    print('Track {}: {}'.format(ii, tracktrack.name))
    for msgmsg in tracktrack:
        print(str(msgmsg))
        MessagesInput2.append(str(msgmsg))

Track 0: Piano 
<meta message track_name name='Piano\x00' time=0>
<meta message time_signature numerator=1 denominator=8 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
<meta message key_signature key='C' time=0>
<meta message set_tempo tempo=545455 time=0>
control_change channel=0 control=121 value=0 time=0
program_change channel=0 program=0 time=0
control_change channel=0 control=7 value=100 time=0
control_change channel=0 control=10 value=64 time=0
control_change channel=0 control=91 value=0 time=0
control_change channel=0 control=93 value=0 time=0
<meta message midi_port port=0 time=0>
note_on channel=0 note=76 velocity=80 time=0
note_on channel=0 note=76 velocity=0 time=113
note_on channel=0 note=75 velocity=80 time=7
note_on channel=0 note=75 velocity=0 time=113
<meta message time_signature numerator=3 denominator=8 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=7>
note_on channel=0 note=76 velocity=80 time=0
note_on channel=0 note=76 velocity=0 time=113
note_on

In [34]:
pygame.init()

pygame.mixer.music.load("output.mid")
pygame.mixer.music.play()